# II. Recommender Systems

## 1. Basics of Recommendation Algorithm

In [32]:
from scipy.spatial.distance import cosine
import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
import pandas as pd
import math

In [10]:
#TODO: load dataset into variable M
import numpy as np

data = [
    [4.0,    3.0,   2.0,    3.0],
    [1.0,    2.0,   3.0,    1.0],
    [np.nan, 2.0,   1.0,    np.nan],
    [4.0,    3.0,   np.nan, np.nan]
]

M = np.array(data)

# Display M
print(M.shape)
print(M)

(4, 4)
[[  4.   3.   2.   3.]
 [  1.   2.   3.   1.]
 [ nan   2.   1.  nan]
 [  4.   3.  nan  nan]]


### Compute similarities

#### Cosine

In [11]:
import math
def cosine_similarity(v1,v2, metric='cosine'):
    #metric: cosine or correlation
    if metric == 'correlation':
        v1 = v1 - np.nanmean(v1)
        v2 = v2 - np.nanmean(v2)
    "compute similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        if np.isnan(x) or np.isnan(y): continue
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def sim_matrix(M, dimension='user', metric='cosine'):
    N = M.shape[0] if dimension == 'user' else M.shape[1]
    sim = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            if i == j:
                sim[i,j] = 0 #Cancel out the effect of self-similarity in the sums later
                continue
            if dimension == 'user':
                v1, v2 = M[i,:], M[j,:]
            else:
                v1, v2 = M[:,i], M[:,j]
            sim[i][j] = cosine_similarity(v1,v2,metric)
    return sim

In [12]:
cosine_similarity(M[0,:], M[2,:], 'cosine')

0.99227787671366774

In [13]:
sim_matrix(M, 'user')

array([[ 0.        ,  0.79582243,  0.99227788,  1.        ],
       [ 0.79582243,  0.        ,  0.86824314,  0.89442719],
       [ 0.99227788,  0.86824314,  0.        ,  1.        ],
       [ 1.        ,  0.89442719,  1.        ,  0.        ]])

In [14]:
sim_matrix(M, 'item')

array([[ 0.        ,  0.9649505 ,  0.73994007,  0.99705449],
       [ 0.9649505 ,  0.        ,  0.90748521,  0.96476382],
       [ 0.73994007,  0.90748521,  0.        ,  0.78935222],
       [ 0.99705449,  0.96476382,  0.78935222,  0.        ]])

#### Pearson

In [15]:
cosine_similarity(M[0,:], M[2,:], 'correlation')

0.70710678118654746

In [16]:
sim_matrix(M, 'user', 'correlation')

array([[ 0.        , -0.85280287,  0.70710678,  0.70710678],
       [-0.85280287,  0.        , -0.5547002 , -0.89442719],
       [ 0.70710678, -0.5547002 ,  0.        , -1.        ],
       [ 0.70710678, -0.89442719, -1.        ,  0.        ]])

In [17]:
sim_matrix(M, 'item', 'correlation')

array([[ 0.        ,  0.94280904, -0.89442719,  0.9486833 ],
       [ 0.94280904,  0.        ,  0.        ,  1.        ],
       [-0.89442719,  0.        ,  0.        , -0.70710678],
       [ 0.9486833 ,  1.        , -0.70710678,  0.        ]])

### a) Compute the missing rating in this table using user-based collaborative filtering (CF). (Use cosine similarity, then use Pearson similarity). Assume taking all neighbors

In [42]:
def user_cf(M, metric='cosine'):
    pred = np.copy(M)
    n_users, n_items = M.shape
    avg_ratings = np.nanmean(M, axis=1)
    sim_users = sim_matrix(M, 'user', metric)
    for i in range(n_users):
        for j in range(n_items):
            if np.isnan(M[i,j]):
                # We are to use the actual formula for this as in the lecture slides
                pred[i,j] = avg_ratings[i] + np.nansum(sim_users[i] * (M[:,j] - avg_ratings)) / sum(sim_users[i])                
                                                    
    return pred

Example of this part of the code:  
<code>np.nansum(sim_users[i] * (M[:,j] - avg_ratings))</code>  

if j = 1:  
sim_users[i] x (M[0,1] - avg_ratings)  
plus    
sim_users[i] x (M[1,1] - avg_ratings)  

In [29]:
print("User-based CF (Cosine): \n" + str(pd.DataFrame(user_cf(M, 'cosine'))))
print("User-based CF (Pearson): \n" + str(pd.DataFrame(user_cf(M, 'correlation'))))


User-based CF (Cosine): 
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2  1.794036  2.0  1.000000  1.272355
3  4.000000  3.0  3.368034  3.268237
User-based CF (Pearson): 
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2  0.764822  2.0  1.000000  1.009169
3  4.000000  3.0  4.616077  2.935013


### b) Similarly, computing the missing rating using item-based CF.

In [24]:
def item_cf(M, metric='cosine'):
    pred = np.copy(M)
    n_users, n_items = M.shape
    avg_ratings = np.nanmean(M, axis=0)
    sim_items = sim_matrix(M, 'item', metric)
    for i in range(n_users):
        for j in range(n_items):
            if np.isnan(M[i,j]):
                # very similar to the previous, just using sim_items now instead
                pred[i,j] = avg_ratings[i] + np.nansum(sim_items[i] * (M[:,j] - avg_ratings)) / sum(sim_items[i])

    return pred

In [25]:
print("Item-based CF (Cosine): \n" + str(pd.DataFrame(item_cf(M, 'cosine'))))
print("Item-based CF (Pearson): \n" + str(pd.DataFrame(item_cf(M, 'correlation'))))

Item-based CF (Cosine): 
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2  2.392903  2.0  1.000000  1.441382
3  4.000000  3.0  1.526011  1.473989
Item-based CF (Pearson): 
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2  3.441518  2.0  1.000000  2.000000
3  4.000000  3.0  2.208141  0.791859


## 2. Evaluating Recommendation Algorithms

### Predictive Accuracy

In [26]:
M_result = np.asarray([[4,3,2,3], 
                [1,2,3,1],
                [1,2,1,2],
                [4,3,2,4]])
pd.DataFrame(M_result)

,0,1,2,3
0,4,3,2,3
1,1,2,3,1
2,1,2,1,2
3,4,3,2,4


In [73]:
def evaluateRS(ratings, groundtruth, method='user_cf', metric='cosine'):
    #method: user_cf and item_cf, metric: cosine and correlation
    if method == 'user_cf':
        prediction = user_cf(ratings, metric)
    else:
        prediction = item_cf(ratings, metric)
    MSE = mean_squared_error(prediction, groundtruth)
    RMSE = round(math.sqrt(MSE),3)
    print("RMSE using {0} approach ({2}) is: {1}".format(method, RMSE, metric))
    # print(pd.DataFrame(prediction))
    return

In [74]:
#TODO: evaluate the predictive accuracy
evaluateRS(M, M_result, 'user_cf', 'cosine')
evaluateRS(M, M_result, 'user_cf', 'correlation')
evaluateRS(M, M_result, 'item_cf', 'cosine')
evaluateRS(M, M_result, 'item_cf', 'correlation')

RMSE using user_cf approach (cosine) is: 0.472
RMSE using user_cf approach (correlation) is: 0.751
RMSE using item_cf approach (cosine) is: 0.744
RMSE using item_cf approach (correlation) is: 1.009


### Ranking Accuracy

In [78]:
import scipy.stats as stats

def evaluate_rank(ratings, groundtruth, method='user_cf', metric='cosine'):
    #metric: cosine vs correlation
    if method == 'user_cf':
        prediction = user_cf(ratings, metric)
    else:
        prediction = item_cf(ratings, metric)
    
    avg_tau = 0
    n_users, n_items = M.shape # need to declare what n_users is
    for i in range(n_users):
        tau, p_value = stats.kendalltau(M_result[i,:], prediction[i,:])
        avg_tau += tau
    avg_tau = avg_tau / n_users
    #clear_output(wait=True)
    print("Rank accuracy using {0} approach ({2}) is: {1}".format(method, avg_tau, metric))
    return

In [79]:
#TODO: calculate the ranking accuracy
evaluate_rank(M, M_result, 'user_cf', 'cosine')
evaluate_rank(M, M_result, 'user_cf', 'correlation')
evaluate_rank(M, M_result, 'item_cf', 'cosine')
evaluate_rank(M, M_result, 'item_cf', 'correlation')

Rank accuracy using user_cf approach (cosine) is: 0.6477056190747297
Rank accuracy using user_cf approach (correlation) is: 0.56719350585564
Rank accuracy using item_cf approach (cosine) is: 0.5456435464587639
Rank accuracy using item_cf approach (correlation) is: 0.5456435464587639
